In [19]:
# Add module folder to notebook
import os
import sys
import pandas

from dotenv import find_dotenv
sys.path.append(os.path.dirname(find_dotenv()))

In [2]:
from snowflake.snowpark import functions
import datetime

from app.snowpark_session.session import snowpark_session

In [3]:
session = snowpark_session()

In [4]:
session.use_database("HOL_DB")

In [39]:
print(f"Current Database and schema: {session.get_fully_qualified_current_schema()}")
print(f"Current Warehouse: {session.get_current_warehouse()}")

Current Database and schema: "HOL_DB"."PUBLIC"
Current Warehouse: "LEIT_WH"


## Explore and Prepare Data

We will use two tables, APPLICATION_RECORD and CREDIT_RECORD, for creating the data set needed for training our model.

In [40]:
# Creating a Snowpark DataFrame
application_record_sdf = session.table('APPLICATION_RECORD')
credit_record_sdf = session.table('CREDIT_RECORD')

### APPLICATION_RECORD

In [41]:
application_record_sdf.count()

438557

#### Remove Duplicates

In [42]:
duplicates_df = application_record_sdf.group_by('ID').agg(functions.count('ID').as_('ID_COUNTS')).filter(functions.col('ID_COUNTS') > 1)
print(f'Total Duplicates: {duplicates_df.count()}')

Total Duplicates: 47


In [43]:
application_record_sdf = application_record_sdf.drop_duplicates('ID')
application_record_sdf.count()

438510

#### Simple Stats


Obtaining simple statistics per column - why are some statistics missing?  
Can you already identify problems in our data?  
For example count always return the number of non null records.  
Hint: ID variables are useless for ML models (but we need it for joining)  
Hint: Missing values for OCCUPATION_TYPE  
Hint: Max Value for DAYS_EMPLOYED seems wrong  

In [44]:
application_record_sdf.describe().to_pandas()

,SUMMARY,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,stddev,5.714962e+05,None,None,None,0.724874,1.100893e+05,None,None,None,None,4185.016222,138770.072835,0.0,0.404523,0.452724,0.310633,None,0.897192
1,count,4.385100e+05,438510,438510,438510,438510.000000,4.385100e+05,438510,438510,438510,438510,438510.000000,438510.000000,438510.0,438510.000000,438510.000000,438510.000000,304317,438510.000000
2,mean,6.022035e+06,None,None,None,0.427381,1.875254e+05,None,None,None,None,-15998.022996,60566.188769,1.0,0.206128,0.287770,0.108200,None,2.194463
3,min,5.008804e+06,F,N,N,0.000000,2.610000e+04,Commercial associate,Academic degree,Civil marriage,Co-op apartment,-25201.000000,-17531.000000,1.0,0.000000,0.000000,0.000000,Accountants,1.000000
4,max,7.999952e+06,M,Y,Y,19.000000,6.750000e+06,Working,Secondary / secondary special,Widow,With parents,-7489.000000,365243.000000,1.0,1.000000,1.000000,1.000000,Waiters/barmen staff,20.000000


#### Missing Value Imputation

The describe output show that we have missing values in OCCUPATION_TYPE, it's count are less (304317) than the total count (438557)  
We will use the `fillna' method to replace missing values in OCCUPATION_TYPE with 'OTHER', since they are so many we creates a new category for it. 

In [45]:
application_record_sdf = application_record_sdf.fillna(value='OTHER', subset=['OCCUPATION_TYPE'])
application_record_sdf.describe().to_pandas()

,SUMMARY,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,count,4.385100e+05,438510,438510,438510,438510.000000,4.385100e+05,438510,438510,438510,438510,438510.000000,438510.000000,438510.0,438510.000000,438510.000000,438510.000000,438510,438510.000000
1,mean,6.022035e+06,None,None,None,0.427381,1.875254e+05,None,None,None,None,-15998.022996,60566.188769,1.0,0.206128,0.287770,0.108200,None,2.194463
2,min,5.008804e+06,F,N,N,0.000000,2.610000e+04,Commercial associate,Academic degree,Civil marriage,Co-op apartment,-25201.000000,-17531.000000,1.0,0.000000,0.000000,0.000000,Accountants,1.000000
3,stddev,5.714962e+05,None,None,None,0.724874,1.100893e+05,None,None,None,None,4185.016222,138770.072835,0.0,0.404523,0.452724,0.310633,None,0.897192
4,max,7.999952e+06,M,Y,Y,19.000000,6.750000e+06,Working,Secondary / secondary special,Widow,With parents,-7489.000000,365243.000000,1.0,1.000000,1.000000,1.000000,Waiters/barmen staff,20.000000


#### Finding constant variables
How many distinct values do we have per column?  
Hint: Constant values are probably irrelevant  
Hint: Variables with many different values can be problematic

In [46]:
unique_values = []
for column in application_record_sdf.columns:
    unique_values.append([column, application_record_sdf.select(column).distinct().count()])
pandas.DataFrame(unique_values, columns=['COLUMN_NAME','NUM_UNIQUE_VALUES'])

,COLUMN_NAME,NUM_UNIQUE_VALUES
0,ID,438510
1,CODE_GENDER,2
2,FLAG_OWN_CAR,2
3,FLAG_OWN_REALTY,2
4,CNT_CHILDREN,12
5,AMT_INCOME_TOTAL,866
6,NAME_INCOME_TYPE,5
7,NAME_EDUCATION_TYPE,5
8,NAME_FAMILY_STATUS,5
9,NAME_HOUSING_TYPE,6


FLAG_MOBIL has only one value so it will not provide any value, so we will drop it from our dataframe.

In [47]:
application_record_sdf = application_record_sdf.drop('FLAG_MOBIL')

#### Response Variable (Target)  
The target variable (column) is in the credit_record_sdf, CREDIT_RECORD, DataFrame and we want to join it with our application_record_sdf dataFrame. Before doing that we will transform the target column.

The traget column, Status, has multiple values which has the following meanings:  
0: 1-29 days past due  
1: 30-59 days past due  
2: 60-89 days overdue  
3: 90-119 days overdue  
4: 120-149 days overdue  
5: Overdue or bad debts, write-offs for more than 150 days  
C: paid off that month  
X: No loan for the month  

In this case we are only interested to flag customers as either high-risk or low-risk. We have defined that a customer is high-risk (1) if during any month they are late on payments by 60 days or more.
All other customers are low-risk (0).

We start by checking how the distribution is among the different status codes.nt status codes.

In [48]:
credit_record_sdf.group_by('STATUS').count().sort('STATUS').to_pandas()

,STATUS,COUNT
0,0,383120
1,1,11090
2,2,868
3,3,320
4,4,223
5,5,1693
6,C,442031
7,X,209230


We will create a new column, TARGET, that will have a 1 for high-risk and 0 for low-risk.

Since credit_record_sdf is on month level we will aggregate and count the number of late payments by customer, ID, and if they have at least one late payment we set TARGET to 1 otherwise to 0.0.

In [49]:
risk_statuses = ['2', '3', '4', '5']
risk_df = credit_record_sdf.group_by('ID') \
    .agg(functions.sum(functions.iff(functions.col('STATUS').in_(risk_statuses), 1, 0)).as_('CNT_LATE')) \
    .with_column('TARGET', functions.when(functions.col('CNT_LATE') > 0, 1).otherwise(0)) \
    .drop('CNT_LATE')

Check the target distribution  
Hint: Highly imbalanced target variables can be problematicc

In [50]:
risk_df.group_by('TARGET').count().to_pandas()

,TARGET,COUNT
0,0,45318
1,1,667


Join risk_df with application_record_sdf

In [54]:
application_record_sdf = application_record_sdf.join(risk_df, using_columns='ID', join_type='inner')
application_record_sdf.count()

36457

In [58]:
application_record_sdf.limit(10).to_pandas()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,TARGET
0,5111190,F,N,Y,2,247500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-13019,-2307,0,0,0,OTHER,4.0,0
1,5140183,F,N,Y,0,67500.0,State servant,Secondary / secondary special,Married,Municipal apartment,-15161,-7190,0,0,0,Core staff,2.0,0
2,5033998,F,N,N,0,112500.0,Commercial associate,Secondary / secondary special,Separated,Municipal apartment,-16417,-1615,0,1,0,Laborers,1.0,0
3,5099974,F,N,N,0,180000.0,Working,Secondary / secondary special,Civil marriage,House / apartment,-10218,-1740,0,1,0,Security staff,2.0,0
4,5095056,F,Y,Y,0,157500.0,Pensioner,Secondary / secondary special,Married,House / apartment,-22684,365243,0,0,0,OTHER,2.0,0
5,5105697,F,Y,N,0,225000.0,Working,Secondary / secondary special,Married,Municipal apartment,-12155,-667,0,0,0,Laborers,2.0,0
6,5024230,F,Y,Y,0,540000.0,Commercial associate,Higher education,Married,House / apartment,-15702,-185,0,1,0,OTHER,2.0,0
7,5029014,F,N,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-13868,-415,0,0,0,Laborers,2.0,0
8,5149342,F,Y,Y,0,292500.0,Commercial associate,Higher education,Single / not married,House / apartment,-16930,-8995,0,0,0,High skill tech staff,1.0,0
9,5061967,F,N,Y,0,225000.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-22897,-1812,0,0,0,Managers,2.0,0


#### Feature Engineering  
For many ML algorithms we need numeric data, so we'll encode our variables.

We will apply the following techniques:  
| COLUMN | TECHNIQUE |  
|---------------------|--------------------------------------------------------|
| FLAG_OWN_CAR | OrdinalEncoder |
| FLAG_OWN_REALTY | OrdinalEncoder |
| AMT_INCOME_TOTAL | StandardScaler |
| CODE_GENDER | OneHotEncoder |
| NAME_INCOME_TYPE | OneHotEncoder |
| NAME_EDUCATION_TYPE | OneHotEncoder |
| NAME_FAMILY_STATUS | OneHotEncoder |
| NAME_HOUSING_TYPE | OneHotEncoder |
| CNT_CHILDREN | Custom Categories + OneHotEncoder |
| CNT_FAM_MEMBERS | Custom Categories + OneHotEncoder |
| OCCUPATION_TYPE | Custom Categories + OneHotEncoder |
| DAYS_BIRTH | Transform to Years + StandardScaler |
| DAYS_EMPLOYED | Transform to Years + Drop Outlier + StandardScaler |
| FLAG_WORK_PHONE | Drop |
| FLAG_PHONE | Drop |
| FLAG_EMAIL | Drop |

#### Children  
Before we decide how many categorise we want for children we check the distribution of the existing values, we also check the number of high-risk customers

In [59]:
var_analysis = application_record_sdf.group_by('CNT_CHILDREN')
var_analysis = var_analysis.agg([
    functions.count('CNT_CHILDREN').as_('COUNT'),
    functions.sum('TARGET').as_('HIGH_RISK'),
])
var_analysis.sort('CNT_CHILDREN').to_pandas()

,CNT_CHILDREN,COUNT,HIGH_RISK
0,0,25201,431
1,1,7492,120
2,2,3256,52
3,3,419,12
4,4,63,1
5,5,20,0
6,7,2,0
7,14,3,0
8,19,1,0


Based on above we will create a new variable/column CNT_CHILDREN_IND with values 0, 1 or 2More and then drop the CNT_CHILDREN column form the dataframe.

In [60]:
application_record_sdf = application_record_sdf.with_column(
    'CNT_CHILDREN_IND',
    functions.iff(functions.col('CNT_CHILDREN') >= 2, '2More', functions.to_varchar(functions.col('CNT_CHILDREN')))
)
application_record_sdf = application_record_sdf.drop('CNT_CHILDREN')
application_record_sdf.limit(10).to_pandas()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,TARGET,CNT_CHILDREN_IND
0,5111190,F,N,Y,247500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-13019,-2307,0,0,0,OTHER,4.0,0,2More
1,5140183,F,N,Y,67500.0,State servant,Secondary / secondary special,Married,Municipal apartment,-15161,-7190,0,0,0,Core staff,2.0,0,0
2,5033998,F,N,N,112500.0,Commercial associate,Secondary / secondary special,Separated,Municipal apartment,-16417,-1615,0,1,0,Laborers,1.0,0,0
3,5099974,F,N,N,180000.0,Working,Secondary / secondary special,Civil marriage,House / apartment,-10218,-1740,0,1,0,Security staff,2.0,0,0
4,5095056,F,Y,Y,157500.0,Pensioner,Secondary / secondary special,Married,House / apartment,-22684,365243,0,0,0,OTHER,2.0,0,0
5,5105697,F,Y,N,225000.0,Working,Secondary / secondary special,Married,Municipal apartment,-12155,-667,0,0,0,Laborers,2.0,0,0
6,5024230,F,Y,Y,540000.0,Commercial associate,Higher education,Married,House / apartment,-15702,-185,0,1,0,OTHER,2.0,0,0
7,5029014,F,N,Y,112500.0,Working,Secondary / secondary special,Married,House / apartment,-13868,-415,0,0,0,Laborers,2.0,0,0
8,5149342,F,Y,Y,292500.0,Commercial associate,Higher education,Single / not married,House / apartment,-16930,-8995,0,0,0,High skill tech staff,1.0,0,0
9,5061967,F,N,Y,225000.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-22897,-1812,0,0,0,Managers,2.0,0,0


In [64]:
application_record_sdf.group_by('CNT_CHILDREN_IND').count().sort('CNT_CHILDREN_IND').to_pandas()

,CNT_CHILDREN_IND,COUNT
0,0,25201
1,1,7492
2,2More,3764
